# Imports

In [108]:
%pylab inline
from bs4 import BeautifulSoup
import urllib2
import subprocess
import requests
import re
import json
import pandas as pd
import pprint
from utils import html_to_pandas
import astroupdate as ad
import datetime
from dateutil.parser import parse
import jinja2
from flask import render_template, render_template_string

Populating the interactive namespace from numpy and matplotlib


/opt/local/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['f']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


# How to get the latest version id for a given software package from astro-update</h3>


In [ ]:
response = urllib2.urlopen('http://heasarc.gsfc.nasa.gov/docs/heasarc/astro-update/')
html=response.read()

In [ ]:
soup=BeautifulSoup(''.join(html))

In [ ]:
table=soup.findAll('table')
soft_table=table[1] # there are 3 tables on the page, the software version table is the 2nd table

In [ ]:
rows = soft_table.findAll('tr')
i=0
name=[]
for row in rows[1:]:
    cols = row.findAll('td')
    i=i+1
    n   =''.join(cols[0].find(text=True))
    name.append(n)
    rep    =''.join(cols[1].find(text=True))
    vers   =''.join(cols[2].find(text=True))
    moddate=''.join(cols[3].find(text=True))
    print "%i %s was last updated by %s to version %s on %s" % (i,n, rep, vers, moddate)
#    progname=''.join(td.find(text=True))
#    print progname

In [ ]:
if 'fv' in name:
    print "CPL in name"

## Python Modules

In [ ]:
import sys

def astroupdate_dict2(url="http://heasarc.gsfc.nasa.gov/docs/heasarc/astro-update/"):
    from bs4 import BeautifulSoup
    import urllib2
    response = urllib2.urlopen('http://heasarc.gsfc.nasa.gov/docs/heasarc/astro-update/')
    html=response.read()
    soup=BeautifulSoup(''.join(html),'lxml')
    table=soup.findAll('table')
    soft_table=table[1] # there are 3 tables on the page, the software version table is the 2nd table
    rows = soft_table.findAll('tr')
    au_dict=dict()
    for row in rows[1:]:
        cols = row.findAll('td')
        n   =''.join(cols[0].find(text=True))
        r   =''.join(cols[1].find(text=True))
        v   =''.join(cols[2].find(text=True))
        m   =''.join(cols[3].find(text=True))
        au_dict[n]=(v.strip(),m.strip(),r.strip())
    return au_dict

def astroupdate_dict(url="http://heasarc.gsfc.nasa.gov/docs/heasarc/astro-update/"):
    from bs4 import BeautifulSoup
    import urllib2
    response = urllib2.urlopen('http://heasarc.gsfc.nasa.gov/docs/heasarc/astro-update/')
    html=response.read()
    soup=BeautifulSoup(''.join(html),'lxml')
    table=soup.findAll('table')
    soft_table=table[1] # there are 3 tables on the page, the software version table is the 2nd table
    rows = soft_table.findAll('tr')
    au_dict=dict()
    for row in rows[1:]:
        cols = row.findAll('td')
        n   =''.join(cols[0].find(text=True))
        r   =''.join(cols[1].find(text=True))
        v   =''.join(cols[2].find(text=True))
        m   =''.join(cols[3].find(text=True))
        homepage = cols[0].findAll('a')[0]['href']
        download = cols[2].findAll('a')[0]['href']
        au_dict[str(n).lower().strip()]={'Version':str(v).strip(),
                                         'Date':str(m).strip(),'Author':str(r).strip(), 
                                         'HomePage':homepage, 'Download':download}
    return au_dict
        
        
def astroupdate(software):
    aud=astroupdate_dict()
    softkey=software.strip().lower()
    try:
        aud[softkey]
    except KeyError, errmsg:
        print "{0} ({1}) not monitored by Astro-Update".format(software, softkey)
        print "Valid entries are:"
        print ' '.join(aud.keys())
        ver, date, author = '','',''
        return ver, date, author
    print "{0} was last updated to version {1} on {2} by {3}".format(software, 
                                                                     aud[softkey]['Version'], 
                                                                     aud[softkey]['Date'], 
                                                                     aud[softkey]['Author'])
    ver = str(aud[softkey]['Version']) 
    date = str(aud[softkey]['Date'])
    author = str(aud[softkey]['Author'])


    return ver, date, author   


        
def auto_update(software):
    """
    for the specified software, notify the user if their installed version is up-to-date; if not,
    give the user the option of downloading the latest version (by taking the user to the
    software download page in their web browser)

    :param software: name of software to check
    :return:
    """
    ad = astroupdate_dict()
    software = software.strip().lower()
    try:
        current_vers = ad[software]['Version']
    except:
        print "{0} not found in Astro-Update Database; returning".format(software)
        return
    updateurl = ad[software]["URL"]

    """
    Compare installed version of software to most recent version
    """

    if software=="heasoft":
        headasdir = os.getenv('HEADAS')
        if not headasdir:
            print "HEASoft not configured (environment variable $HEADAS is not defined); stopping"
            return
        try:
            fver_installed = subprocess.check_output(['fversion'])
        except:
            print "fversion failed; is HEASoft installed?"
            sys.exit()
        vers=fver_installed.strip("\n").split('_V')
        vers=vers[1].rstrip()
        print "Latest version of  HEASoft = %s; You currently have HEASoft version %s" % (current_vers, vers)
        if current_vers.strip()<> vers.strip():
            ans=''
            ans=raw_input("Would you like to update (Y/n)? ")
            if ans.strip()=='' or ans[0].lower()=='y':
                print "Opening HEASoft download page in your web browser"
                #webbrowser.open('http://heasarc.gsfc.nasa.gov/docs/software/lheasoft/download.html')
                webbrowser.open(updateurl)

    if software=='sae':
        fermidir = os.getenv('FERMI_DIR')
        if not fermidir:
            print "SAE not configured (environment variable $FERMI_DIR is not defined); stopping"
            return
        STools = glob.glob(fermidir+'/../../ScienceTools*') # Find Science Tools directory
        #print STools
        foundversion = False
        if not STools:
            print "Problem finding {0}; Is SAE installed?".format(fermidir+'/../..')
            return
        vers = STools[0].split('Tools-')[1].split('-fssc')[0].strip()
        #print SToolsver
        print "The current version of the Science Analysis Environment for Fermi is {0}; you have version {1}".format(current_vers.strip(), vers.strip())
        if current_vers.strip()<>vers.strip():
            ans=raw_input("Would you like to update (Y/n)? ")
            if ans.strip()=='' or ans[0].lower()=='y':
                print "Opening Fermi SAE download page in your web browser"
                #webbrowser.open('http://fermi.gsfc.nasa.gov/ssc/data/analysis/software/')
                webbrowser.open(updateurl)

    if software =='xspec':
        headasdir = os.getenv('HEADAS')
        foundversion = False
        if not headasdir:
            print "XSPEC not configured (environment variable $HEADAS is not defined); stopping"
            return
        """
        Get installed primary version number from locally installed manual.html; this is defined as a constant in
        the file $HEADAS/../Xspec/src/XSUtil/Utils/XSutility.cxx in a line like this: static const string version = "12.9.0b";
        """
        xsutil=headasdir+'/../Xspec/src/XSUtil/Utils/XSutility.cxx'
        f=open(xsutil,'r')
        for line in f.readlines():
            #print line
            if 'version =' in line:
                #print line
                vers = line.split('=')[1].strip().split('"')[1]
                #print 'Vers = {0}'.format(vers.strip())
                foundversion=True
                break
        f.close()
        if not foundversion:
            print "Could not find local XSpec version; stopping"
            return
        print "The current version of XSpec is {0}; you have version {1}".format(current_vers.strip(), vers.strip())
        if current_vers.strip()<>vers.strip():
            ans=raw_input("Would you like to update (Y/n)? ")
            if ans.strip()=='' or ans[0].lower()=='y':
                print "Opening the XSpec download page in your web browser"
                #webbrowser.open('http://fermi.gsfc.nasa.gov/ssc/data/analysis/software/')
                webbrowser.open(updateurl)

    if software == 'ciao':
        ascds = os.getenv('ASCDS_INSTALL')
        if not ascds:
            print "CIAO not configured (environment variable $ASCDS_INSTALL is not defined); stopping"
            return
        os.system(ascds+'/contrib/bin/check_ciao_version')


    if software == 'sas':
        sasdir = os.getenv('SAS_DIR')
        if not sasdir:
            print "SAS not configured (environment variable $SAS_DIR is not defined); stopping"
            return
        vers = sasdir.split('sas_')[1].split('/')[0]
        print "The current version of SAS is {0}; you have version {1}".format(current_vers.strip(), vers.strip())
        if current_vers.strip() <> vers.strip():
            ans = raw_input("Would you like to update (Y/n)? ")
            if ans.strip() == '' or ans[0].lower() == 'y':
                print "Opening SAS download page in your web browser"
                # webbrowser.open('http://fermi.gsfc.nasa.gov/ssc/data/analysis/software/')
                webbrowser.open(updateurl)

    return


In [ ]:
aud = astroupdate_dict()
aud['saoimage ds9']

In [ ]:
software = 'HEASOFT'
ver, date, org = astroupdate(software)

In [ ]:
# aud = astroupdate_dict()
key ='xxx'
astroupdate('xxx')

In [ ]:
software='fv'
astroupdate(software)

# Determining the current version of a Specified Software Package

For a specified software package, we want:
    * to read the web page which has the version info
    * extract the version info from the web page
    * and (eventually) compare this version to the one in astro-update
    

## Let's Try SPEX

In [ ]:
#
# Take SPEX as an example
# idea is to find some text on the page which signals the version 
#  (something like "Current Version is ")
# locate that text on the page then extract the version
# this may have to be done uniquely for each software package
# but let's try to find the text "Version" initially
#

spex_version_page = 'https://www.sron.nl/astrophysics-spex/download'
#spex_version_id = 'version' # text on the page which indicates the version
spex_version_pattern = r'\d+.\d+.\d+' # the pattern for the version numbering scheme
pat = re.compile(spex_version_pattern)

req = requests.get(spex_version_page)
ver = pat.search(req.text)

try:
    print "The Current SPEX version is {0}".format(ver.group(0))
except:
    print "couldn't find current version of SPEX at"
    print spex_version_page
    
req.close()

In [ ]:
ver, date, org = astroupdate('spex')

## Fermi SAE

In [ ]:
version_page = 'http://fermi.gsfc.nasa.gov/ssc/data/analysis/software/'
#spex_version_id = 'version' # text on the page which indicates the version
version_pattern = r'v\d+r\d+p\d+' # v10r0p5 for example
pat = re.compile(version_pattern)

req = requests.get(version_page)
ver = pat.search(req.text)

try:
    print "The Current SAE version is {0}".format(ver.group(0))
except:
    print "couldn't find current version of SPEX at"
    print spex_version_page
    
req.close()

In [ ]:
aud = astroupdate_dict()
version_pattern = {}
for k in aud.keys():
    print k, aud[k]['Version']
    pat = raw_input('Specify regular expression for version > ')
    version_pattern[k] = pat

## General method of Finding current version of software 

In [ ]:
#
# here's how to find the current version of the HEASoft.  Looks like we need a search key (like 
# 'Current version' for the HEASoft) or to better define the search regular expression
#
spattern = json.load(open('resources/astro_update_version_patterns.txt'))
pat = re.compile(spattern['heasoft'])

aud = astroupdate_dict()
req = requests.get(aud['heasoft']['Download'])

skey = 'Current version'
ind = req.text.index(skey)

req.text[ind:ind+100]
ver = pat.search(req.text[ind:ind+100])
print "Current version of the HEASoft is {0}".format(ver.group(0))


In [ ]:
#
# a more robust search - using xspec as an example
#
#
# this should do it - we need to add a "version_marker" phrase for each software package
# 

spattern = json.load(open('resources/astro_update_version_patterns.txt'))

aud['xspec']
aud['xspec']['version_marker'] = 'latest update'

req = requests.get(aud['xspec']['Download'])

soup = BeautifulSoup(req.text)
# note - use soup.text to get find the version_marker to avoid embedded html
ind = soup.text.index(aud['xspec']['version_marker']) 
spattern['xspec'] = '\d+.\\d+.\\d+[a-z]'
req.close()
pat = re.compile(spattern['xspec']) 
pat.search(soup.text[ind:]).group() # only search the string after the version_marker has been found


In [ ]:
sinfo = json.load(open('resources/astro_update_sinfo.txt'))

sinfo.keys()

keys = sinfo.keys()

for k in keys:
    print k
    pat = re.compile(sinfo[k]['pattern'])

    aud = astroupdate_dict()
    req = requests.get(aud[k]['Download'])

    skey = sinfo[k]['version_marker']
    skip = False
    try: 
        ind = req.text.index(skey)
    except: 
        print "{0} not found on {1}".format(skey, aud[k]['Download'])
        skip = True
    
    if not skip:
        req.text[ind:]
        ver = pat.search(req.text[ind:ind+100])
        req.close()
        try:
            print "Current version of the {1} is {0}; expected {2}".format(ver.group(0), k, aud[k]['Version'])
        except:
            print "Version for {0} not found on {1}".format(k, aud[k]['Download'])
        
        



# How to find the latest version for a CALDB</h2>

we define a caldb python dictionary as follows:

caldb={'ASCA':{'GIS':20060711,'SIS':19991111,'XRT':19991111,'MIS':19940114}, 'BEPPOSAX':{'HGSPC':1999, 'LECS':1999, 'MECS':1999, 'PDS':1999, 'WFC':1999} ...}

this dictionary should be defined by parsing the "supported missions" webpage
http://heasarc.gsfc.nasa.gov/docs/heasarc/caldb/caldb_supported_missions.html

In [ ]:
def caldb_dict(url="http://heasarc.gsfc.nasa.gov/docs/heasarc/caldb/caldb_supported_missions.html"):
    """
    Parse the caldb supported missions web page and convert to a python library
    """
    from bs4 import BeautifulSoup
    import urllib2
    response = urllib2.urlopen(url)
    html=response.read()
    soup=BeautifulSoup(''.join(html))
    table=soup.findAll('table')
    hcal_table=table[1] # there are 3 tables on the page, the caldb version table is the 2nd table
    rows = hcal_table.findAll('tr')
    #a= ''.join(cols.find(text=True)).strip()
    #print a
    caldb_dict=dict()
    """
    Read first row - contains the first mission
       if first cell in row is not empty: mission name=mis
    """
    cols = rows[0].findAll('td')
    mis =str(cols[0].find(text=True)).strip().lower()
    #print "Found %s" % mis
    """
    Read next row
       check that first cell in row is empty: 
         if cell empty, then add to instrument dictionary
            inst_dict[cell[1]]=cell[2] (i.e. something like inst_dict={'XIS':'20140101'}) 
                 where cell[1] is the instrument name (XIS) and cell[2] is the version date 
          if cell is not empty: found new mission
               so store instr_dict in caldb_dict for previous mission: CALDB_DICT[MIS]=inst_dict
               set mis = new mission
               clear inst_dict
               repeat
    """
    inst_dict=dict()
    for row in rows[1:]: # omit first row (i.e. row 0)
        cols = row.findAll('td')
        #print cols[0]
        a=cols[0].find(text=True)
        #print a
        if len(a.strip()) == 0: # no entry in mission cell, so these are instruments
            inst= str(cols[1].find(text=True)).strip().lower()
            vers= str(cols[2].find(text=True)).strip()
            #print mis, inst, vers
            try:
                inst_dict[inst]=caldate2vers(vers)
            except:
                inst_dict[inst]=vers
        else: # save instruments & version in mission dictionary
            #print inst_dict
            #print "current mission is %s" % mis
            caldb_dict[mis]=inst_dict
            #print caldb_dict
            inst_dict=dict()
            mis=str(a.strip()).lower()
            #print "Found new mission %s " % mis
    return caldb_dict

def caldate2vers(update_date):
    """
    converts latest update dates (a string in mmm 00, yyyy format) to yyyymmdd format
    """
    month=({'Apr':'04','Aug':'08','Dec':'12','Feb':'02','Jan':'01','Jul':'07','Jun':'06','Mar':'03','May':'05','Nov':'11',
          'Oct':'10','Sep':'09'})
    a=update_date.split(',')
    yyyy=a[1].strip()
    b=a[0].split()
    mon=b[0]
    day=b[1]
    return yyyy+month[mon]+day
    
def current_caldb_vers(mis, inst):
    import ftplib
    caldir="caldb/data/"+mis.lower().strip()+"/"+inst.lower().strip()
    ftp = ftplib.FTP("heasarc.gsfc.nasa.gov")
    ftp.login("anonymous", "")
    ftpmess=ftp.cwd(caldir)
    list=[]
    ftp.dir('caldb.indx', list.append)
    ftp.close()
    vers=list[0].split()[-1].split('/')[1].split('indx')[-1] #[-1] is the last element
    return vers

In [ ]:
caldict=caldb_dict()

In [ ]:
mis='rxte'
inst='pca'
print caldict[mis][inst]
cur_vers=current_caldb_vers(mis, inst)
print cur_vers
if caldict['nustar']['fpm']==cur_vers:
    print "You're Current!"
else:
    print "Not current :-("

In [ ]:
caldict['rxte']

In [ ]:
a=caldict['ASCA']['GIS'].split(',')
yyyy=a[1].strip()
yyyy
a[0]
a[0].split()
c='1999'+'11'+'11'
c

print caldict['ASCA']['GIS']
print caldate2vers('Nov 11, 1999')

In [ ]:
cal_dict

In [ ]:
caldict['ASCA']

In [ ]:
a={'b':'c',2:5}
a
a.clear()
a

In [ ]:
from bs4 import BeautifulSoup
import urllib2
url="http://heasarc.gsfc.nasa.gov/docs/heasarc/caldb/caldb_supported_missions.html"
response = urllib2.urlopen(url)
html=response.read()
soup = BeautifulSoup(html)
table=soup.findAll('table')
hcal_table=table[1]# there are 3 tables on the page, the software version table is the 2nd table
rows = hcal_table.findAll('tr')
cols = rows[3].findAll('td')
mis =str(cols[0].find(text=True))

In [ ]:
import calendar
enumerate(calendar.month_abbr)
dict((v,k) for k,v in enumerate(calendar.month_abbr))

In [ ]:
mon['Sep']

In [ ]:
import ftplib
ftp = ftplib.FTP("heasarc.gsfc.nasa.gov")
ftp.login("anonymous", "")
ftpmess=ftp.cwd("caldb/data/nustar/fpm")
list=[]
ftp.dir('caldb.indx', list.append)
ftp.close()
list

In [ ]:
a=list[0].split()

In [ ]:
a[-1]

In [ ]:
vers=list[0].split()[-1].split('/')[1].split('indx')[-1]

In [ ]:
vers

a is {{13.5/2.0}}

# JSON files for each software package

Let's create a json file for each software package, for example 

```/software/github/heasarc/astro-update/astroupdate_resources/ds9_au.json```

to hold information needed to determine the version number of the software package
and also to store the link information

Note: this could  probably work but it requires that each software page have it's own regular expression, and it's difficult to construct regular expressions for each software package that will be general enough ("7.5" vs "7.5.1" for example)

## this create a dataframe from the json dictionary for a specified software package.  

In printciple this method could be used to create the html data for the astroupdate page

In [ ]:
# get defs for ds9 and POA
jds9 = "/software/github/heasarc/astro_update/definitions/ds9_au.json"
d = json.load(open(jds9, mode='r'))
d['saoimage ds9'].keys()
testdf = pd.DataFrame.from_dict(data=d, orient='index')
jpoa = "/software/github/heasarc/astro_update/definitions/poa_au.json"
dpoa = json.load(open(jpoa, mode='r'))


In [ ]:
jaud = "/software/github/heasarc/astro_update/definitions/astroupdate_defs.json"
daud = json.load(open(jaud, mode='r'))
testdf  = pd.DataFrame.from_dict(daud, orient='index')
testdf

In [ ]:
testdf.loc['poa'].version_url
req = requests.get(testdf.loc['poa'].version_url)
alesoup = BeautifulSoup(req.text,'lxml')
testdf.loc['poa'].pattern_marker
ind = alesoup.text.lower().find('the most recent version')
re.compile(testdf.loc['poa'].pattern).search(alesoup.text[ind:ind+100]).group()

In [ ]:
try:
    testdf.loc['PINT of ALE'].version_url
except KeyError, errmsg:
    print "{0}".format(errmsg)
    print('Available software keys')
    for x in testdf.index:
        print x,


In [ ]:
testdf.keys()


In [ ]:
req = requests.get(d['saoimage ds9']['version_url'])

In [ ]:
ind = req.text.index(d['saoimage ds9']['pattern_marker'])
pat=re.compile(d['saoimage ds9']['pattern'])

In [ ]:
m = pat.search(req.text,ind, ind+100)

In [ ]:
req.text[ind:ind+200]

In [ ]:
txt = u'SAOImage DS9 Version 7.5.1</span>'

In [ ]:
pat = re.compile("\\d\.\\d?\.?\\d?")
m = pat.search(txt)
m.group()

# A Crude Alternative

would be to have a script that would read the astro_update dictionary, display the current version, then display the software web page showing the current version of the software package; the user could then enter a yes or no if the astro-update info for that package needed to be updated 

In [ ]:
import webbrowser

In [ ]:
for soft in aud.keys():
    print "Astro-Update Version of {1} = {0}".format(aud[soft]['Version'], soft)
    print "Display Version page for {0}".format(soft)
    webbrowser.open(aud[soft]['HomePage'])
    ans = raw_input('Enter Version (<0 to ignore) > ')

In [ ]:
aud['heasoft']

# Using Pandas to parse astro-update html table

Pandas is useful to sort the table by version date - but pandas "read_html" function seems to strip out the links from the columns and just return the column text.  How can we keep the links?

What I'd like to do:
* get the html data with requests.get()
* parse the html with BeautifulSoup and get the table of interest
* either add a row (or rows) or update  existing row/rows
* sort the table by time (descending)
* create an updated astroupdate.html page with the updated table

In [ ]:
# get html with requests
req = requests.get('http://heasarc.gsfc.nasa.gov/docs/heasarc/astro-update')
# create soup and extract table
soup = BeautifulSoup(req.text)
autab = soup('table')[1]
# update row or add new row
# 
# then create dataframe
audf = pd.read_html(str(autab))[0]
# sort dataframe by "Last Update"
auddf['Last Update'] = pd.to_datetime(auddf['Last Update'])
auddf.head()
auddfsort = auddf.sort_values(by = 'Last Update', ascending=False)
nsoup = BeautifulSoup(auddf.to_html())
nsoup('tr')[2]('td')

In [ ]:
test= list(autab('tr')[1]('td'))
reshape(test,(1,4))
dftest = pd.DataFrame(data=reshape(test,(1,4)), columns = ['Name','resp','version','update'])
print(dftest['Name'])

In [ ]:
a=str(dftest['Name'][0])
s = BeautifulSoup(a)
s

In [ ]:
auddf = aud[1]
auddf.head()

## sorting the dataframe by time

In [ ]:
auddf['Last Update'] = pd.to_datetime(auddf['Last Update'])
auddf.sort_values(by='Last Update')

# Using BeautifulSoup to parse astro-update html table

let's create astro-update definition files from the current (2017-01-31) astro-update table

In [ ]:
req = requests.get('http://heasarc.gsfc.nasa.gov/docs/heasarc/astro-update/')

In [ ]:
ausoup = BeautifulSoup(req.text,'lxml')

In [ ]:
tab = ausoup.findAll('table')

In [ ]:
rows = tab[1].find_all('tr')

In [ ]:
for r in rows[1:]: # skip 1st (header) row
    col = r.findAll('td')
    name = str(col[0].text.split(':')[0].strip().lower())
    try:
        desc = str(col[0].text.split(':')[1]).strip()
    except IndexError:
        print "Could not get description for {0}".format(name)
        desc = name
    homepage = str(col[0].a.get('href'))
    auth = str(col[1].text)
    ver = str(col[2].text)
    download = str(col[2].a.get('href'))
    updated = str(col[3].text)
    # note - json needs strings to have double, not single, quotes
    audict = {"{0}".format(name):{"pattern":"\\d+\\.{0,1}\\d+",
                                    "pattern_description":"one or more digits followed by zeror or 1 period followed by one or more digits",
                                    "pattern_marker":"version",
                                    "home_url":"{0}".format(homepage),
                                    "download_url":"{0}".format(download),
                                    "version_url":"{0}".format(download),
                                    "description":"{0}".format(desc),
                                    "author":"{0}".format(auth),
                                    "author_page":"{0}".format(homepage)}}
    fdir = '/software/github/heasarc/astro_update/definitions/TEST'
    fname = "{0}/{1}_au.json".format(fdir,name)
    try:
        with open(fname, mode='w') as fout:
            json.dump(audict, fout, indent=4)
    except IOError, errmsg:
        print errmsg



In [ ]:
print audict

## Create an astroupdate row


In [ ]:
req = requests.get('http://heasarc.gsfc.nasa.gov/docs/heasarc/astro-update/')
ausoup = BeautifulSoup(req.text,'lxml')
tab = ausoup.findAll('table')
#cols = tab[1].find_all('tr')[3].find_all('td')
cols = tab[1]('tr')[3]('td')
cols[0]

In [ ]:
aud = json.load(open('definitions/TEST/pyraf_au.json'))

In [ ]:
cols[0].a.next_sibling

# Retaining links in an html table using pd.read_html

pd.read_html only returns the text from an html table without the links; you need to do some work to maintain the links

see http://stackoverflow.com/questions/31771619/html-table-to-pandas-table-info-inside-html-tags

20170202: the util.html_to_pandas function solves this problem!

In [ ]:
req = requests.get('http://heasarc.gsfc.nasa.gov/docs/heasarc/astro-update/')
audtable = BeautifulSoup(req.text,'lxml')('table')[1]


In [ ]:
auddf[auddf.columns[0]][0]


In [ ]:
# see http://stackoverflow.com/questions/26277757/pandas-to-html-truncates-string-contents
pd.set_option('display.max_colwidth', -1)
auddf['sortdate'] = pd.to_datetime(auddf['Last Update'])
auddf.sort_values(by="sortdate", ascending=True, inplace=True)
auddf[auddf.columns[0]].iloc[0]

soup = BeautifulSoup(auddf.to_html(escape=False),'lxml')('table')[0]
print(soup.prettify())

In [ ]:
auddf = ad.read_au_defs()

In [ ]:
auddf.loc['poa'].pattern

In [ ]:
auddf.loc['poa'].name

In [ ]:
auddf.loc[auddf.index[0]]


# Update and output the astro-update defs database

In [ ]:
adj  = json.load(open('definitions/astroupdate_defs.json','r'))

In [ ]:
adj['browse']['ad_version'] = '0.9'

In [ ]:
print json.dumps(adj,indent=4) # this prints a nicely formatted json dictionary

# output the AU dataframe as a json file 

to be done after updating the software version and release date, for example

In [ ]:
# Can we pretty print the pandas dataframe as a json file?

apd = ad.read_au_defs()
apd.loc['cfitsio'].ad_version = '1.0'
apdj = apd.to_json(orient='index')
p=json.loads(apdj)
print json.dumps(p,indent=4) # this prints a nicely formatted json dictionary

## Sort the dataframe by time

see http://stackoverflow.com/questions/28161356/sort-pandas-dataframe-by-date

In [95]:
#
# Example
#
pdf = pd.DataFrame( {'Symbol':['A','F','C'] ,
    'Date':['12/14/16','12/15/16','08/21/14']})
pdf.Date = pd.to_datetime(pdf.Date)
pdf.sort_values(by='Date', ascending=False)

,Date,Symbol
1,2016-12-15,F
0,2016-12-14,A
2,2014-08-21,C


In [96]:
apd = ad.read_au_defs()
aud = ad.read_au_defs()

In [97]:
apd.ad_release_date = pd.to_datetime(apd.ad_release_date, errors="coerce")
apd.sort_values(by='ad_release_date', ascending=False, inplace=True)
apd[['ad_release_date']]

,ad_release_date
xamin,2017-02-14
tempo2,2017-02-14
casa,2017-02-08
vltpipe,2017-01-27
apt,2017-01-26
heasoft,2017-01-18
aladin,2017-01-17
xspec,2017-01-17
visit,2017-01-16
pimms,2017-01-16


# Rendering the web page as a static html file

see https://realpython.com/blog/python/primer-on-jinja-templating/

and http://kagerato.net/articles/software/libraries/jinja-quickstart.html

In [99]:
templateLoader = jinja2.FileSystemLoader( searchpath="/software/github/heasarc/astro_update/templates" )
templateEnv = jinja2.Environment( loader=templateLoader )
template = templateEnv.get_template('test_table.html')

In [100]:
audef = ad.read_au_defs()
# convert release date to a datetime object...
audef.ad_release_date = pd.to_datetime(audef.ad_release_date, errors="coerce")
# ... then sort by release_date
audef.sort_values(by='ad_release_date', ascending=False, inplace=True)
# strip out the rows without a defined release_date
audef_for_web = audef[audef.ad_release_date.notnull()]


In [101]:
output_text = template.render(audef= audef_for_web)

In [105]:
f = open('/software/github/heasarc/astro_update/html/astroupdate_test.html', mode='w')

In [106]:
f.write(output_text)

In [107]:
f.close()


In [109]:
aud = ad.read_au_defs()


In [112]:
aud.loc['xamin']['name']


u'XAMIN'

In [111]:
aud

,release_pattern,release_description,description,release_url,author,pattern,release_marker,ad_version,download_url,version_url,pattern_marker,pattern_description,author_page,ad_release_date,home_url,name
acis_extract,\d{2}\-\D{3}\-\d{4},MM/DD/YY,IDL software for analysis of Chandra/ACIS data,http://www.astro.psu.edu/xray/docs/TARA/code,Broos et al.,\d{4}\D{3}\d{2},,2016sep22,http://www.astro.psu.edu/xray/docs/TARA/code,http://www.astro.psu.edu/xray/docs/TARA/code,,a 4-digit year followed by the 3-letter month abbreviation followed a 2-digit date,http://www2.astro.psu.edu/xray/,09/22/16,http://www2.astro.psu.edu/xray/docs/TARA/ae_users_guide.html,ACIS Extract
aips,\d{2}-\D{3}-\d{4},dd-mon-yyyy,Astronomical Image Processing System,http://www.aips.nrao.edu/cook.html,NRAO,\d{2}-\D{3}-\d{4},VERSION FOR,31-DEC-2016,http://www.aips.nrao.edu/index.shtml#CURRENT_RELEASE,http://www.aips.nrao.edu/cook.html,VERSION FOR,dd-mon-yyyy,http://www.aips.nrao.edu/index.shtml,12/31/16,http://www.aips.nrao.edu/index.shtml,AIPS
aladin,"\d{1,2}/\d{1,2}/\d{4}","Dec 15, 2016",Desktop Observatory to visualize and manipulate astronomical images,http://aladin.u-strasbg.fr/java/nph-aladin.pl?frame=downloading#Beta,CDS,"v\d\.{0,1}\d{0,4}",,v9.504,http://aladin.u-strasbg.fr/AladinDesktop/#Download,http://aladin.u-strasbg.fr/java/nph-aladin.pl?frame=downloading#Beta,Beta version v,a 'v' followed by a number followed by a dot followed by a three-digit number,http://cds.u-strasbg.fr/,01/17/17,http://aladin.u-strasbg.fr/AladinDesktop/,Aladin
amuse,\d{2}/\d{2}/\d{2},MM/DD/YY,Astrophysical Multipurpose Software Environment for simulations,http://www.amusecode.org,Leiden,"\d+\.{0,1}\d+.{0,1}\d+",release date:,10.0,http://www.amusecode.org,http://www.amusecode.org,Current release:,"one or more digits followed by zeror or 1 period followed by one or more digits,followed by zeror or 1 period followed by one or more digits",https://groups.google.com/forum/#!forum/amusecode,09/17/14,http://www.amusecode.org,AMUSE
apt,"[a-zA-Z]+\s{1}\d{1,2},\s{1}\d{4}","(Full month name) (1-2 digit day of month), (4 digit year)",Astronomers Proposal Tool,http://www.stsci.edu/hst/proposing/apt,STScI,"\d+\.{0,1}\d+\.{0,1}\d+",was released on,25.0.3,http://www.stsci.edu/hst/proposing/apt,http://www.stsci.edu/hst/proposing/apt,Release Information,one or more digits followed by zeror or 1 period followed by one or more digits,http://www.stsci.edu/hst/proposing/apt,01/26/17,http://www.stsci.edu/hst/proposing/apt,APT
astrogrid,\d{2}-\D{3}-\d{4},"(Full month name) (1-2 digit day of month), (4 digit year)",Virtual Observatory Software for Astronomers,http://www.astrogrid.org/,UK VOP,\d{4}-\d{4},last updated:,2001-2010,http://www.astrogrid.org/,http://www.astrogrid.org/,developement project from,one or more digits followed by zeror or 1 period followed by one or more digits,http://www.astrogrid.org/,08/15/12,http://www.astrogrid.org/,AstroGrid
atomdb,"[a-z,A-Z]+\s\d{1}\D{2}\s{1}\d{4}","Full name of month, 1-2 digit day, 2 character for st, th, etc, four digit year",atomic database for X-ray plasma spectral modeling,http://atomdb.org,CXC,"\d{1,2}\.{0,1}\d+\.\d",News,3.0.8,http://atomdb.org,http://atomdb.org,News,one or more digits followed by zeror or 1 period followed by one or more digits,http://atomdb.org,12/08/16,http://atomdb.org,ATOMDB
browse,"[a-z,A-Z]+\s\d{1},\s{1}\d{4}","Full name of month, 1-2 digit day, four digit year",Old-school access to the HEASARC catalogs and data,http://heasarc.gsfc.nasa.gov/W3Browse/version-history.html,HEASARC,v\d+\.+\d+,,v8.4,http://heasarc.gsfc.nasa.gov/W3Browse/,http://heasarc.gsfc.nasa.gov/W3Browse/version-history.html,,a 'v' followed by one or more numbers followed by 0 or one dot followed by 0 or more numbers,http://heasarc.gsfc.nasa.gov,10/09/07,http://heasarc.gsfc.nasa.gov/W3Browse/,Browse
browse_extract,\d{4}-\d{2}-\d{2},"four digit year, dash, two digit month, dash, two digit day",command-line query of HEASARC databases,http://heasarc.gsfc.nasa.gov/FTP